## Import all necessary libraries

In [15]:
import numpy as np
import random as rd
import cv2
import glob
from skimage import io
from matplotlib import pyplot as plt
import pandas as pd

# Function HOG
We start by defining a HOG() function that computes the Histogram of Oriented Gradients of any 64x128 image.
The function takes as parameter the path to said image and returns a 3708 elements vector corresponding to the HOG of the function.

In [16]:
def HOG(path):
    img = cv2.imread(path)             # It is necessary here to start by normalizing the image
    img = np.float32(img) / 255.0     # All the values are between 0 and 1
    kernelX = np.array([[0, 0, 0],
                        [-1, 0, 1],    #Here we design two kernels,  
                        [0, 0, 0]])    # one to get the "horizontal" derivate of the gradient
    kernelY = np.array([[0, -1, 0],                        # (derivate with regard to X)
                        [0, 0, 0],     # And the other one to get the "vertical" derivate of the gradient
                        [0, 1, 0]])                                   # (derivate with regard to Y)
    tempX = cv2.filter2D(img, -1, kernelX) # We convolute the image with the two kernels 
    tempY = cv2.filter2D(img, -1, kernelY)    # And set the result in grayscale
    gX = cv2.cvtColor(tempX, cv2.COLOR_BGR2GRAY)  # Indeed the color information is not relevant for us
    gY = cv2.cvtColor(tempY, cv2.COLOR_BGR2GRAY)   # We get gX and gY the gradient image with regard to X and Y
    Norm, Angle = cv2.cartToPolar(gX, gY, angleInDegrees=True)
    # This function here allows, using gX and gY, to obtain the Norm and Angle of the gradient.
    # It exploits the formulas : Norm = sqrt(gX^2 + gY^2)  and Angle = atan(gY/gX)  (to apply to every pixel)
    for i in range(128):
        for j in range(64):              # The Angle matrix contains value between 0° and 360°
            if Angle[i, j] >= 180:       # Such a wide range of values is useless, therefore we reduce it here to 
                Angle[i, j] = Angle[i, j] - 180                 # values between 0° (included) and 180° (excluded)
    
    # We now have the orientation and norm values of the gradient for each pixel
    # The next step is to separate the image into 8x8 cells and compute an histogram of oriented gradients for each
    cellsHOG = np.array([[[0.0]*9]*8]*16)
    # We have 16 rows of 8 cells, and for each one the histogram can be represented as a 9-element vector
    # Each element corresponding to a discrete orientation of the gradient.
    discAngle = np.array([0, 20, 40, 60, 80, 100, 120, 140, 160])
    # This vector represents the 9 discrete values for the orientation of the gradient.
    for j in range(0, 128, 8):
        for i in range(0, 64, 8):       # Here we are simply creating the 8x8 cells, for both Norm and Angle matrices
            cellNorm = Norm[j:j+8, i:i+8]
            cellAngle = Angle[j:j+8, i:i+8]
            hogCell = np.array([0.0]*9)   # For each cell we represent the histogram as a 9-element vector
            for x in range(8):
                for y in range(8): # (for each pixel in the cell and for each orientation value)
                    for k in range(1, 8):
                    # Each pixel's orientation value is between two discrete orientation values.
                    # We split the norm value of the gradient between each discrete orientation value,
                    # according to the distance between the 2 discrete values and the real one (the one of the pixel).
                    # For example, if the real orientation is 10° and the norm is 1, we will put 0.5 in the spot
                    # corresponding to 0° and 0.5 in the spot corresponding to 10°.
                        if (discAngle[k-1] <= cellAngle[x, y]) and (cellAngle[x, y] <= discAngle[k]) :
                            hogCell[k-1] = hogCell[k-1] + (cellAngle[x, y] - discAngle[k-1])*(cellNorm[x, y]/20)
                            hogCell[k] = hogCell[k] + (discAngle[k] - cellAngle[x, y])*(cellNorm[x, y]/20)
                    # The case where the angle is bigger than 160° is treated almost the same way:
                    # It means the angle is between 160° and 180° (180° being the max as we assured before)
                    # So we consider 180° as we would consider 0°, and split the norm value between the 160° spot
                    # and the 0° spot, according to the distances computed as before, using 180° instead of 0°.
                    if cellAngle[x, y] > discAngle[8]:
                        hogCell[8] = hogCell[8] + (cellAngle[x, y] - discAngle[8])*(cellNorm[x, y]/20)
                        hogCell[0] = hogCell[0] + (180 - cellAngle[x, y])*(cellNorm[x, y]/20)
            cellsHOG[j//8, i//8] = hogCell # hogCell represents the hog for the current cell
            #cellsHOG is a matrix that contains the HOG of each cell, sorted by position of the cell in the image
    
    # For the last step we need to design 16x16 overlapping blocks (each containing 2x2 cells)
    # (each cell is present in one block at least and four blocks at most)
    blocksNormalHOG = np.array([])
    for k in range(len(cellsHOG)-1):
        for h in range(len(cellsHOG[0])-1):   # Here we design those blocks
            vec = np.array([])
            for b in range(k, k+2):   # Each block has 4 cells, so it has four 9-element vectors representing a HOG.
                for c in range(h, h+2):  # We concatenate these 4 vectors for each block
                    vec = np.concatenate((vec, cellsHOG[b, c]))   # And we get a 36-element vector for each block.
            vec = vec/np.linalg.norm(vec) # We normalize all these 36-element vectors.
            blocksNormalHOG = np.concatenate((blocksNormalHOG, vec))
            # And then we just have to concatenate them.
            # We had 8x16 cells, which means (since we use overlapping blocks of 2x2 cells) that we have 7x15 blocks.
            # The final vector we are returning has then 7x15x36 = 3780 elements.
    
    return blocksNormalHOG

# Creating the dataframe using the HOG() function
We will use here all the images from the training and test datasets, compute their HOG, labeling them 0 or 1 depending on if they contain humans or not (1 corresponding to the case where they contain a human), and put them in different dataframes.

In [20]:
# We start by creating an empty training dataset
train_data = pd.DataFrame(data=None, columns=["HOG", "label"])

# Then we look at all the images containing humans in the training dataset
train_pos_mask = "Datasets\\Train\\Pos\\*.*"
train_pos_names = glob.glob(train_pos_mask)     # We use the same tool as during the pre-processing
for path in train_pos_names:
    print("processing %s..." % path,)
    data_i = pd.Series([HOG(path), 1], index=train_data.columns)
    train_data = train_data.append(data_i, ignore_index=True) # We add the computed data to the dataframe object
    # As they are all images containing humans, we label them 1
# Now we do the same thing for the images not containing humans in the training dataset
train_neg_mask = "Datasets\\Train\\Neg\\*.*"
train_neg_names = glob.glob(train_neg_mask)
for path in train_neg_names:
    print("processing %s..." % path,)
    data_i = pd.Series([HOG(path), 0], index=train_data.columns)
    train_data = train_data.append(data_i, ignore_index=True)
    # As they are all images not containing humans, we label them 0
train_data = train_data.sample(frac=1).reset_index(drop=True)  #We shuffle the dataframe and reset the index
# The shuffling is necessary because of the way we built this dataset (all the pos then all the neg)
train_data.to_csv("Datasets\\Train\\train_data.csv")
# And we save this dataframe into a csv

processing Datasets\Train\Pos\crop001001a.jpg...
processing Datasets\Train\Pos\crop001001b.jpg...
processing Datasets\Train\Pos\crop001001c.jpg...
processing Datasets\Train\Pos\crop001001d.jpg...
processing Datasets\Train\Pos\crop001001e.jpg...
processing Datasets\Train\Pos\crop001001f.jpg...
processing Datasets\Train\Pos\crop001002a.jpg...
processing Datasets\Train\Pos\crop001002b.jpg...
processing Datasets\Train\Pos\crop001002c.jpg...
processing Datasets\Train\Pos\crop001002d.jpg...
processing Datasets\Train\Pos\crop001003a.jpg...
processing Datasets\Train\Pos\crop001003b.jpg...
processing Datasets\Train\Pos\crop001003c.jpg...
processing Datasets\Train\Pos\crop001003d.jpg...
processing Datasets\Train\Pos\crop001004a.jpg...
processing Datasets\Train\Pos\crop001004b.jpg...
processing Datasets\Train\Pos\crop001004c.jpg...
processing Datasets\Train\Pos\crop001004d.jpg...
processing Datasets\Train\Pos\crop001005a.jpg...
processing Datasets\Train\Pos\crop001005b.jpg...
processing Datasets\

processing Datasets\Train\Pos\crop001058a.jpg...
processing Datasets\Train\Pos\crop001058b.jpg...
processing Datasets\Train\Pos\crop001059a.jpg...
processing Datasets\Train\Pos\crop001059b.jpg...
processing Datasets\Train\Pos\crop001060a.jpg...
processing Datasets\Train\Pos\crop001060b.jpg...
processing Datasets\Train\Pos\crop001061a.jpg...
processing Datasets\Train\Pos\crop001061b.jpg...
processing Datasets\Train\Pos\crop001061c.jpg...
processing Datasets\Train\Pos\crop001061d.jpg...
processing Datasets\Train\Pos\crop001063a.jpg...
processing Datasets\Train\Pos\crop001063b.jpg...
processing Datasets\Train\Pos\crop001063c.jpg...
processing Datasets\Train\Pos\crop001063d.jpg...
processing Datasets\Train\Pos\crop001063e.jpg...
processing Datasets\Train\Pos\crop001063f.jpg...
processing Datasets\Train\Pos\crop001063g.jpg...
processing Datasets\Train\Pos\crop001063h.jpg...
processing Datasets\Train\Pos\crop001064a.jpg...
processing Datasets\Train\Pos\crop001064b.jpg...
processing Datasets\

processing Datasets\Train\Pos\crop001103a.jpg...
processing Datasets\Train\Pos\crop001103b.jpg...
processing Datasets\Train\Pos\crop001104a.jpg...
processing Datasets\Train\Pos\crop001104b.jpg...
processing Datasets\Train\Pos\crop001104c.jpg...
processing Datasets\Train\Pos\crop001104d.jpg...
processing Datasets\Train\Pos\crop001104e.jpg...
processing Datasets\Train\Pos\crop001104f.jpg...
processing Datasets\Train\Pos\crop001105a.jpg...
processing Datasets\Train\Pos\crop001105b.jpg...
processing Datasets\Train\Pos\crop001106a.jpg...
processing Datasets\Train\Pos\crop001106b.jpg...
processing Datasets\Train\Pos\crop001107a.jpg...
processing Datasets\Train\Pos\crop001107b.jpg...
processing Datasets\Train\Pos\crop001107c.jpg...
processing Datasets\Train\Pos\crop001107d.jpg...
processing Datasets\Train\Pos\crop001107e.jpg...
processing Datasets\Train\Pos\crop001107f.jpg...
processing Datasets\Train\Pos\crop001108a.jpg...
processing Datasets\Train\Pos\crop001108b.jpg...
processing Datasets\

processing Datasets\Train\Pos\crop001146a.jpg...
processing Datasets\Train\Pos\crop001146b.jpg...
processing Datasets\Train\Pos\crop001147a.jpg...
processing Datasets\Train\Pos\crop001147b.jpg...
processing Datasets\Train\Pos\crop001148a.jpg...
processing Datasets\Train\Pos\crop001148b.jpg...
processing Datasets\Train\Pos\crop001149a.jpg...
processing Datasets\Train\Pos\crop001149b.jpg...
processing Datasets\Train\Pos\crop001149c.jpg...
processing Datasets\Train\Pos\crop001149d.jpg...
processing Datasets\Train\Pos\crop001150a.jpg...
processing Datasets\Train\Pos\crop001150b.jpg...
processing Datasets\Train\Pos\crop001151a.jpg...
processing Datasets\Train\Pos\crop001151b.jpg...
processing Datasets\Train\Pos\crop001152a.jpg...
processing Datasets\Train\Pos\crop001152b.jpg...
processing Datasets\Train\Pos\crop001153a.jpg...
processing Datasets\Train\Pos\crop001153b.jpg...
processing Datasets\Train\Pos\crop001154a.jpg...
processing Datasets\Train\Pos\crop001154b.jpg...
processing Datasets\

processing Datasets\Train\Pos\crop001265e.jpg...
processing Datasets\Train\Pos\crop001265f.jpg...
processing Datasets\Train\Pos\crop001265g.jpg...
processing Datasets\Train\Pos\crop001265h.jpg...
processing Datasets\Train\Pos\crop001267a.jpg...
processing Datasets\Train\Pos\crop001267b.jpg...
processing Datasets\Train\Pos\crop001267c.jpg...
processing Datasets\Train\Pos\crop001267d.jpg...
processing Datasets\Train\Pos\crop001267e.jpg...
processing Datasets\Train\Pos\crop001267f.jpg...
processing Datasets\Train\Pos\crop001267g.jpg...
processing Datasets\Train\Pos\crop001267h.jpg...
processing Datasets\Train\Pos\crop001267i.jpg...
processing Datasets\Train\Pos\crop001267j.jpg...
processing Datasets\Train\Pos\crop001268a.jpg...
processing Datasets\Train\Pos\crop001268b.jpg...
processing Datasets\Train\Pos\crop001268c.jpg...
processing Datasets\Train\Pos\crop001268d.jpg...
processing Datasets\Train\Pos\crop001268e.jpg...
processing Datasets\Train\Pos\crop001268f.jpg...
processing Datasets\

...

processing Datasets\Train\Neg\no_person__no_bike_202_patch_8_.png...
processing Datasets\Train\Neg\no_person__no_bike_202_patch_9_.png...
processing Datasets\Train\Neg\no_person__no_bike_204_patch_0_.png...
processing Datasets\Train\Neg\no_person__no_bike_204_patch_1_.png...
processing Datasets\Train\Neg\no_person__no_bike_204_patch_2_.png...
processing Datasets\Train\Neg\no_person__no_bike_204_patch_3_.png...
processing Datasets\Train\Neg\no_person__no_bike_204_patch_4_.png...
processing Datasets\Train\Neg\no_person__no_bike_204_patch_5_.png...
processing Datasets\Train\Neg\no_person__no_bike_204_patch_6_.png...
processing Datasets\Train\Neg\no_person__no_bike_204_patch_7_.png...
processing Datasets\Train\Neg\no_person__no_bike_204_patch_8_.png...
processing Datasets\Train\Neg\no_person__no_bike_204_patch_9_.png...
processing Datasets\Train\Neg\no_person__no_bike_205_patch_0_.png...
processing Datasets\Train\Neg\no_person__no_bike_205_patch_1_.png...
processing Datasets\Train\Neg\no_p

processing Datasets\Train\Neg\no_person__no_bike_220_patch_7_.png...
processing Datasets\Train\Neg\no_person__no_bike_220_patch_8_.png...
processing Datasets\Train\Neg\no_person__no_bike_220_patch_9_.png...
processing Datasets\Train\Neg\no_person__no_bike_222_patch_0_.png...
processing Datasets\Train\Neg\no_person__no_bike_222_patch_1_.png...
processing Datasets\Train\Neg\no_person__no_bike_222_patch_2_.png...
processing Datasets\Train\Neg\no_person__no_bike_222_patch_3_.png...
processing Datasets\Train\Neg\no_person__no_bike_222_patch_4_.png...
processing Datasets\Train\Neg\no_person__no_bike_222_patch_5_.png...
processing Datasets\Train\Neg\no_person__no_bike_222_patch_6_.png...
processing Datasets\Train\Neg\no_person__no_bike_222_patch_7_.png...
processing Datasets\Train\Neg\no_person__no_bike_222_patch_8_.png...
processing Datasets\Train\Neg\no_person__no_bike_222_patch_9_.png...
processing Datasets\Train\Neg\no_person__no_bike_223_patch_0_.png...
processing Datasets\Train\Neg\no_p

processing Datasets\Train\Neg\no_person__no_bike_238_patch_6_.png...
processing Datasets\Train\Neg\no_person__no_bike_238_patch_7_.png...
processing Datasets\Train\Neg\no_person__no_bike_238_patch_8_.png...
processing Datasets\Train\Neg\no_person__no_bike_238_patch_9_.png...
processing Datasets\Train\Neg\no_person__no_bike_240_patch_0_.png...
processing Datasets\Train\Neg\no_person__no_bike_240_patch_1_.png...
processing Datasets\Train\Neg\no_person__no_bike_240_patch_2_.png...
processing Datasets\Train\Neg\no_person__no_bike_240_patch_3_.png...
processing Datasets\Train\Neg\no_person__no_bike_240_patch_4_.png...
processing Datasets\Train\Neg\no_person__no_bike_240_patch_5_.png...
processing Datasets\Train\Neg\no_person__no_bike_240_patch_6_.png...
processing Datasets\Train\Neg\no_person__no_bike_240_patch_7_.png...
processing Datasets\Train\Neg\no_person__no_bike_240_patch_8_.png...
processing Datasets\Train\Neg\no_person__no_bike_240_patch_9_.png...
processing Datasets\Train\Neg\no_p

processing Datasets\Train\Neg\no_person__no_bike_256_patch_5_.png...
processing Datasets\Train\Neg\no_person__no_bike_256_patch_6_.png...
processing Datasets\Train\Neg\no_person__no_bike_256_patch_7_.png...
processing Datasets\Train\Neg\no_person__no_bike_256_patch_8_.png...
processing Datasets\Train\Neg\no_person__no_bike_256_patch_9_.png...
processing Datasets\Train\Neg\no_person__no_bike_258_patch_0_.png...
processing Datasets\Train\Neg\no_person__no_bike_258_patch_1_.png...
processing Datasets\Train\Neg\no_person__no_bike_258_patch_2_.png...
processing Datasets\Train\Neg\no_person__no_bike_258_patch_3_.png...
processing Datasets\Train\Neg\no_person__no_bike_258_patch_4_.png...
processing Datasets\Train\Neg\no_person__no_bike_258_patch_5_.png...
processing Datasets\Train\Neg\no_person__no_bike_258_patch_6_.png...
processing Datasets\Train\Neg\no_person__no_bike_258_patch_7_.png...
processing Datasets\Train\Neg\no_person__no_bike_258_patch_8_.png...
processing Datasets\Train\Neg\no_p

,HOG,label
0,"[0.07733569199378448, 0.17508420301101596, 0.0...",1
1,"[0.17419552828508908, 0.1873917000405962, 0.09...",0
2,"[0.11382955653981809, 0.0785782952764628, 0.03...",0
3,"[0.1484476952678407, 0.3213385205161816, 0.145...",0
4,"[0.10410055665218575, 0.17956561224005987, 0.0...",0
...,...,...
14591,"[0.33211776866832105, 0.6405549421902834, 0.04...",0
14592,"[0.0, 0.049348039488968244, 0.0375762692207218...",0
14593,"[0.0, 0.48937747829259776, 0.1384229930718696,...",0
14594,"[0.0, 0.34095433920744805, 0.0, 0.0, 0.0227302...",0


In [19]:
# We now do the exact same thing to create the test dataset
# We start by creating an empty testing dataset
test_data = pd.DataFrame(data=None, columns=["HOG", "label"])

# Then we look at all the images containing humans in the test dataset
test_pos_mask = "Datasets\\Test\\Pos\\*.*"
test_pos_names = glob.glob(test_pos_mask)     # We use the same tool as during the pre-processing
for path in test_pos_names:
    print("processing %s..." % path,)
    data_i = pd.Series([HOG(path), 1], index=test_data.columns)
    test_data = test_data.append(data_i, ignore_index=True)
    # As they are all images containing humans, we label them 1
# Now we do the same thing for the images not containing humans in the test dataset
test_neg_mask = "Datasets\\Test\\Neg\\*.*"
test_neg_names = glob.glob(test_neg_mask)
for path in test_neg_names:
    print("processing %s..." % path,)
    data_i = pd.Series([HOG(path), 0], index=test_data.columns)
    test_data = test_data.append(data_i, ignore_index=True)
    # As they are all images not containing humans, we label them 0

test_data = test_data.sample(frac=1).reset_index(drop=True)  #We shuffle the dataframe and reset the index
# The shuffling is necessary because of the way we built this dataset (all the pos then all the neg)
test_data.to_csv("Datasets\\Test\\test_data.csv")
#And we save this dataframe into a csv

processing Datasets\Test\Pos\crop001501a.jpg...
processing Datasets\Test\Pos\crop001501b.jpg...
processing Datasets\Test\Pos\crop001501c.jpg...
processing Datasets\Test\Pos\crop001501d.jpg...
processing Datasets\Test\Pos\crop001501e.jpg...
processing Datasets\Test\Pos\crop001501f.jpg...
processing Datasets\Test\Pos\crop001501g.jpg...
processing Datasets\Test\Pos\crop001501h.jpg...
processing Datasets\Test\Pos\crop001504a.jpg...
processing Datasets\Test\Pos\crop001504b.jpg...
processing Datasets\Test\Pos\crop001504c.jpg...
processing Datasets\Test\Pos\crop001504d.jpg...
processing Datasets\Test\Pos\crop001511a.jpg...
processing Datasets\Test\Pos\crop001511b.jpg...
processing Datasets\Test\Pos\crop001512a.jpg...
processing Datasets\Test\Pos\crop001512b.jpg...
processing Datasets\Test\Pos\crop001512c.jpg...
processing Datasets\Test\Pos\crop001512d.jpg...
processing Datasets\Test\Pos\crop001512e.jpg...
processing Datasets\Test\Pos\crop001512f.jpg...
processing Datasets\Test\Pos\crop001514a

processing Datasets\Test\Pos\crop001654b.jpg...
processing Datasets\Test\Pos\crop001658a.jpg...
processing Datasets\Test\Pos\crop001658b.jpg...
processing Datasets\Test\Pos\crop001658c.jpg...
processing Datasets\Test\Pos\crop001658d.jpg...
processing Datasets\Test\Pos\crop001658e.jpg...
processing Datasets\Test\Pos\crop001658f.jpg...
processing Datasets\Test\Pos\crop001659a.jpg...
processing Datasets\Test\Pos\crop001659b.jpg...
processing Datasets\Test\Pos\crop001660a.jpg...
processing Datasets\Test\Pos\crop001660b.jpg...
processing Datasets\Test\Pos\crop001661a.jpg...
processing Datasets\Test\Pos\crop001661b.jpg...
processing Datasets\Test\Pos\crop001661c.jpg...
processing Datasets\Test\Pos\crop001661d.jpg...
processing Datasets\Test\Pos\crop001664a.jpg...
processing Datasets\Test\Pos\crop001664b.jpg...
processing Datasets\Test\Pos\crop001664c.jpg...
processing Datasets\Test\Pos\crop001664d.jpg...
processing Datasets\Test\Pos\crop001664e.jpg...
processing Datasets\Test\Pos\crop001664f

processing Datasets\Test\Pos\crop_000026h.jpg...
processing Datasets\Test\Pos\crop_000027a.jpg...
processing Datasets\Test\Pos\crop_000027b.jpg...
processing Datasets\Test\Pos\crop_000027c.jpg...
processing Datasets\Test\Pos\crop_000027d.jpg...
processing Datasets\Test\Pos\crop_000604a.jpg...
processing Datasets\Test\Pos\crop_000604b.jpg...
processing Datasets\Test\Pos\crop_000604c.jpg...
processing Datasets\Test\Pos\crop_000604d.jpg...
processing Datasets\Test\Pos\crop_000604e.jpg...
processing Datasets\Test\Pos\crop_000604f.jpg...
processing Datasets\Test\Pos\person_007a.jpg...
processing Datasets\Test\Pos\person_007b.jpg...
processing Datasets\Test\Pos\person_008a.jpg...
processing Datasets\Test\Pos\person_008b.jpg...
processing Datasets\Test\Pos\person_009a.jpg...
processing Datasets\Test\Pos\person_009b.jpg...
processing Datasets\Test\Pos\person_010a.jpg...
processing Datasets\Test\Pos\person_010b.jpg...
processing Datasets\Test\Pos\person_011a.jpg...
processing Datasets\Test\Pos\

processing Datasets\Test\Pos\person_132e.jpg...
processing Datasets\Test\Pos\person_132f.jpg...
processing Datasets\Test\Pos\person_132g.jpg...
processing Datasets\Test\Pos\person_132h.jpg...
processing Datasets\Test\Pos\person_132i.jpg...
processing Datasets\Test\Pos\person_132j.jpg...
processing Datasets\Test\Pos\person_132k.jpg...
processing Datasets\Test\Pos\person_132l.jpg...
processing Datasets\Test\Pos\person_132m.jpg...
processing Datasets\Test\Pos\person_132n.jpg...
processing Datasets\Test\Pos\person_134a.jpg...
processing Datasets\Test\Pos\person_134b.jpg...
processing Datasets\Test\Pos\person_134c.jpg...
processing Datasets\Test\Pos\person_134d.jpg...
processing Datasets\Test\Pos\person_134e.jpg...
processing Datasets\Test\Pos\person_134f.jpg...
processing Datasets\Test\Pos\person_134g.jpg...
processing Datasets\Test\Pos\person_134h.jpg...
processing Datasets\Test\Pos\person_135a.jpg...
processing Datasets\Test\Pos\person_135b.jpg...
processing Datasets\Test\Pos\person_135c

processing Datasets\Test\Pos\person_210f.jpg...
processing Datasets\Test\Pos\person_212a.jpg...
processing Datasets\Test\Pos\person_212b.jpg...
processing Datasets\Test\Pos\person_216a.jpg...
processing Datasets\Test\Pos\person_216b.jpg...
processing Datasets\Test\Pos\person_216c.jpg...
processing Datasets\Test\Pos\person_216d.jpg...
processing Datasets\Test\Pos\person_217a.jpg...
processing Datasets\Test\Pos\person_217b.jpg...
processing Datasets\Test\Pos\person_217c.jpg...
processing Datasets\Test\Pos\person_217d.jpg...
processing Datasets\Test\Pos\person_217e.jpg...
processing Datasets\Test\Pos\person_217f.jpg...
processing Datasets\Test\Pos\person_217g.jpg...
processing Datasets\Test\Pos\person_217h.jpg...
processing Datasets\Test\Pos\person_217i.jpg...
processing Datasets\Test\Pos\person_217j.jpg...
processing Datasets\Test\Pos\person_218a.jpg...
processing Datasets\Test\Pos\person_218b.jpg...
processing Datasets\Test\Pos\person_218c.jpg...
processing Datasets\Test\Pos\person_218d

...

processing Datasets\Test\Neg\no_person__no_bike_144_patch_9_.png...
processing Datasets\Test\Neg\no_person__no_bike_148_patch_0_.png...
processing Datasets\Test\Neg\no_person__no_bike_148_patch_1_.png...
processing Datasets\Test\Neg\no_person__no_bike_148_patch_2_.png...
processing Datasets\Test\Neg\no_person__no_bike_148_patch_3_.png...
processing Datasets\Test\Neg\no_person__no_bike_148_patch_4_.png...
processing Datasets\Test\Neg\no_person__no_bike_148_patch_5_.png...
processing Datasets\Test\Neg\no_person__no_bike_148_patch_6_.png...
processing Datasets\Test\Neg\no_person__no_bike_148_patch_7_.png...
processing Datasets\Test\Neg\no_person__no_bike_148_patch_8_.png...
processing Datasets\Test\Neg\no_person__no_bike_148_patch_9_.png...
processing Datasets\Test\Neg\no_person__no_bike_151_patch_0_.png...
processing Datasets\Test\Neg\no_person__no_bike_151_patch_1_.png...
processing Datasets\Test\Neg\no_person__no_bike_151_patch_2_.png...
processing Datasets\Test\Neg\no_person__no_bike_

processing Datasets\Test\Neg\no_person__no_bike_185_patch_0_.png...
processing Datasets\Test\Neg\no_person__no_bike_185_patch_1_.png...
processing Datasets\Test\Neg\no_person__no_bike_185_patch_2_.png...
processing Datasets\Test\Neg\no_person__no_bike_185_patch_3_.png...
processing Datasets\Test\Neg\no_person__no_bike_185_patch_4_.png...
processing Datasets\Test\Neg\no_person__no_bike_185_patch_5_.png...
processing Datasets\Test\Neg\no_person__no_bike_185_patch_6_.png...
processing Datasets\Test\Neg\no_person__no_bike_185_patch_7_.png...
processing Datasets\Test\Neg\no_person__no_bike_185_patch_8_.png...
processing Datasets\Test\Neg\no_person__no_bike_185_patch_9_.png...
processing Datasets\Test\Neg\no_person__no_bike_188_patch_0_.png...
processing Datasets\Test\Neg\no_person__no_bike_188_patch_1_.png...
processing Datasets\Test\Neg\no_person__no_bike_188_patch_2_.png...
processing Datasets\Test\Neg\no_person__no_bike_188_patch_3_.png...
processing Datasets\Test\Neg\no_person__no_bike_

processing Datasets\Test\Neg\no_person__no_bike_221_patch_1_.png...
processing Datasets\Test\Neg\no_person__no_bike_221_patch_2_.png...
processing Datasets\Test\Neg\no_person__no_bike_221_patch_3_.png...
processing Datasets\Test\Neg\no_person__no_bike_221_patch_4_.png...
processing Datasets\Test\Neg\no_person__no_bike_221_patch_5_.png...
processing Datasets\Test\Neg\no_person__no_bike_221_patch_6_.png...
processing Datasets\Test\Neg\no_person__no_bike_221_patch_7_.png...
processing Datasets\Test\Neg\no_person__no_bike_221_patch_8_.png...
processing Datasets\Test\Neg\no_person__no_bike_221_patch_9_.png...
processing Datasets\Test\Neg\no_person__no_bike_224_patch_0_.png...
processing Datasets\Test\Neg\no_person__no_bike_224_patch_1_.png...
processing Datasets\Test\Neg\no_person__no_bike_224_patch_2_.png...
processing Datasets\Test\Neg\no_person__no_bike_224_patch_3_.png...
processing Datasets\Test\Neg\no_person__no_bike_224_patch_4_.png...
processing Datasets\Test\Neg\no_person__no_bike_

processing Datasets\Test\Neg\no_person__no_bike_257_patch_2_.png...
processing Datasets\Test\Neg\no_person__no_bike_257_patch_3_.png...
processing Datasets\Test\Neg\no_person__no_bike_257_patch_4_.png...
processing Datasets\Test\Neg\no_person__no_bike_257_patch_5_.png...
processing Datasets\Test\Neg\no_person__no_bike_257_patch_6_.png...
processing Datasets\Test\Neg\no_person__no_bike_257_patch_7_.png...
processing Datasets\Test\Neg\no_person__no_bike_257_patch_8_.png...
processing Datasets\Test\Neg\no_person__no_bike_257_patch_9_.png...
processing Datasets\Test\Neg\no_person__no_bike_260_patch_0_.png...
processing Datasets\Test\Neg\no_person__no_bike_260_patch_1_.png...
processing Datasets\Test\Neg\no_person__no_bike_260_patch_2_.png...
processing Datasets\Test\Neg\no_person__no_bike_260_patch_3_.png...
processing Datasets\Test\Neg\no_person__no_bike_260_patch_4_.png...
processing Datasets\Test\Neg\no_person__no_bike_260_patch_5_.png...
processing Datasets\Test\Neg\no_person__no_bike_

,HOG,label
0,"[0.0011165242594885906, 0.023510209190648765, ...",0
1,"[0.07980871637406235, 0.09829627246146005, 0.0...",0
2,"[0.0, 0.028359004209759012, 0.0052666136279376...",0
3,"[0.009230686024815898, 0.05228747416918726, 0....",0
4,"[0.03434648940853972, 0.049714612620177484, 0....",1
...,...,...
5651,"[0.014552234903384406, 0.08364306095972183, 0....",0
5652,"[0.013693529969097616, 0.026824315251025004, 0...",0
5653,"[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...",0
5654,"[0.12079318072764035, 0.10455897817455569, 0.0...",0
